In [1]:
reset -fs

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
from pyproj import CRS
import folium


## Example polygon

In [3]:
# lat_point_list = [50.854457, 52.518172, 50.072651, 48.853033, 50.854457]
# lon_point_list = [4.377184, 13.407759, 14.435935, 2.349553, 4.377184]

# polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
# crs = CRS("WGS84")
# polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
# print(polygon.geometry)

# polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
# polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")

In [4]:
# m = folium.Map([50.854457, 4.377184], zoom_start=5, tiles='cartodbpositron')
# folium.GeoJson(polygon).add_to(m)
# folium.LatLngPopup().add_to(m)
# m

## Loading solar eclipse path

In [5]:
eclipse_path = pd.read_csv('solar_eclipse_2024_04_08.csv')
eclipse_path.head()

,UTC,NLAT,NLON,SLAT,SLON,CLAT,CLON
0,16:42,05 30.6S,149 47.6W,06 11.7S,146 38.0W,05 50.2S,148 07.8W
1,16:44,04 20.5S,145 29.6W,05 08.4S,143 00.6W,04 44.0S,144 13.0W
2,16:46,03 21.2S,142 27.6W,04 12.3S,140 15.6W,03 46.4S,141 20.3W
3,16:48,02 27.1S,140 01.8W,03 20.2S,137 59.5W,02 53.3S,138 59.7W
4,16:50,01 36.2S,137 58.5W,02 30.8S,136 02.5W,02 03.3S,136 59.7W


In [6]:
# convert UTC to datetime
eclipse_path['UTC'] = pd.to_datetime('2024-04-08 ' + eclipse_path['UTC'], format='%Y-%m-%d %H:%M')


In [7]:
#function to convert DDM (degree decimal minutes) to decimal degrees

import re

def ddm_to_dec(ddm_str): 
    sign = -1 if re.search('[swSW]', ddm_str) else 1
    numbers = [*filter(len, re.split('\D+', ddm_str))]
    degree = numbers[0]
    minute_decimal = numbers[1] 
    decimal_val = numbers[2] if len(numbers) > 2 else '0' 
    minute_decimal += "." + decimal_val

    return sign * (int(degree) + float(minute_decimal) / 60)



In [8]:
eclipse_path.loc[:,'NLAT':'CLON'] = eclipse_path.loc[:,'NLAT':'CLON'].applymap(ddm_to_dec)

## Map the path using folium

In [9]:
lat_point_list = []
lon_point_list = []

lat_point_list.extend(eclipse_path['NLAT'].tolist())
lat_point_list.extend(eclipse_path['SLAT'].values[::-1].tolist())
lat_point_list.append(eclipse_path['NLAT'].iloc[0].tolist())

lon_point_list.extend(eclipse_path['NLON'].tolist())
lon_point_list.extend(eclipse_path['SLON'].values[::-1].tolist())
lon_point_list.append(eclipse_path['NLON'].iloc[0].tolist())

path_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = CRS("WGS84")
path = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[path_geom])       
print(path.geometry)

path.to_file(filename='path.geojson', driver='GeoJSON')
#polygon.to_file(filename='path.shp', driver="ESRI Shapefile")

0    POLYGON ((-149.79333 -5.51000, -145.49333 -4.3...
Name: geometry, dtype: geometry


In [10]:
path_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')

folium.GeoJson(path).add_to(path_map)
folium.LatLngPopup().add_to(path_map)
path_map

## Creating boxes for selecting weather station data

In [ ]:
box1_lat = [23.6, 28.2, 27.0, 22.4, 23.6]
box1_lon = [-106.9, -102.5, -101.1, -105.6, -106.9]

box2_lat = [27.0, 28.2, 32.9, 31.5, 27.0]
box2_lon = [-101.1, -102.5, -97.3, -96.1, -101.1]

box3_lat = [31.5, 32.9, 36.9, 35.5, 31.5]
box3_lon = [-96.1, -97.3, -91.9, -91.0, -96.1]

box1_geom = Polygon(zip(box1_lon, box1_lat))
crs = CRS("WGS84")
box1 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[box1_geom])

box2_geom = Polygon(zip(box2_lon, box2_lat))
crs = CRS("WGS84")
box2 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[box2_geom])

box3_geom = Polygon(zip(box3_lon, box3_lat))
crs = CRS("WGS84")
box3 = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[box3_geom])

print(box3.geometry)

In [ ]:
path_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')

folium.GeoJson(path).add_to(path_map)
folium.GeoJson(box1).add_to(path_map)
folium.GeoJson(box2).add_to(path_map)
folium.GeoJson(box3).add_to(path_map)
folium.LatLngPopup().add_to(path_map)
path_map

In [ ]:
station_test = pd.read_csv('station_test.csv')
station_test.head()

In [ ]:
station_test.shape

In [ ]:
station_test.columns = [column.strip() for column in station_test.columns]
station_test.columns

In [ ]:
pd.set_option('display.max_rows', 800)
station_test = station_test.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# station_test['SUBPVDR'].str.isspace()

In [ ]:
# usgs = (station_test.groupby(['STAID','SUBPVDR','LAT','LON'],as_index=False))
usgs = station_test[(station_test['SUBPVDR'] == 'USGS')]
usgs.head()


In [ ]:
usgs.shape

In [ ]:
stations = [Point(xy) for xy in zip(usgs['LON'], usgs['LAT'])]
stations[:3]

In [ ]:
usgs_stations = gpd.GeoDataFrame(usgs, crs = CRS("WGS84"), geometry = stations)
usgs_stations.head()

In [ ]:
usgs_json = usgs_stations.to_json()

In [ ]:
station_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')
points = folium.features.GeoJson(usgs_json)
# zip_codes = folium.features.GeoJson('tx_texas_zip_codes_geo.min.json')

station_map.add_child(points)
# station_map.add_child(zip_codes)

folium.GeoJson(path).add_to(station_map)
folium.LatLngPopup().add_to(station_map)
station_map

## NWS weather stations 

In [20]:
world_stations = pd.read_csv('world_stations.csv', skiprows=4)
world_stations.head()

/Users/noether/miniforge3/envs/eclipse/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3251: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,country3,country2,country,region,subregion,place_name,station_name,type,stn_key,status,icao,national_id,wmo,wban,ghcn,special,lat,lon,elev
0,ABW,AW,Aruba,-,-,Old Norwood,John A. Osborne AP,NaN,AWaaNORW,-,NaN,NaN,78850.0,NaN,NaN,NaN,16.791111,-62.193333,166.7
1,ABW,AW,Aruba,SA,-,Upper Hell's Gate,"Juancho E. Yrausquin AP, Saba",NaN,AWaaHLGT,-,NaN,NaN,78871.0,NaN,NaN,NaN,17.646111,-63.220833,42.1
2,AFG,AF,Afghanistan,BAL,-,Kyzylabad|Mazar-I-Sharif|Qizil?b?d,Kyzylabad,NaN,AFaaOAMS,-,OAMS,NaN,40911.0,NaN,NaN,NaN,36.700000,67.200000,392.3
3,AFG,AF,Afghanistan,BAL,-,Pushti-Bag|Mazari Sharif|Dehd?d?,Camp Spann,NaN,AFaaKQSP,-,KQSP,NaN,NaN,NaN,NaN,NaN,36.650400,66.996000,408.0
4,AFG,AF,Afghanistan,BAL,-,Qala-i-Gul Mohd|Mazari Sharif|Qizil?b?d,Camp Marmal,NaN,AFaaKQML,-,KQML,NaN,NaN,NaN,NaN,NaN,36.703000,67.228000,391.0


In [21]:
usa_stations = world_stations[(world_stations['country3'] == 'USA')]

In [22]:
usa_stations.head(40)

,country3,country2,country,region,subregion,place_name,station_name,type,stn_key,status,icao,national_id,wmo,wban,ghcn,special,lat,lon,elev
31913,USA,US,United States,NaN,NaN,"Albany, Ny.","Albany, Ny.",NaN,DZsALBAN,-,NaN,NaN,72518.0,NaN,NaN,NaN,42.691944,-73.832222,95.0
31914,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99201,-,NaN,NaN,99201.0,NaN,NaN,DB/201 CMAN/41001,34.900000,-72.900000,NaN
31915,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99203,-,NaN,NaN,99203.0,NaN,NaN,DB/203 CMAN/46001,56.300000,-148.300000,3.0
31916,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99204,-,NaN,NaN,99204.0,NaN,NaN,DB/204 CMAN/42001,25.900000,-89.700000,NaN
31917,USA,US,United States,-,-,Buoy,Hotel,NaN,USaaKC7H,-,KC7H,C7H,99207.0,NaN,NaN,DB/ CMAN/44004,38.500000,-70.700000,NaN
31918,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99215,-,NaN,NaN,99215.0,NaN,NaN,DB/215 CMAN/41002,32.300000,-75.300000,NaN
31919,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99216,-,NaN,NaN,99216.0,NaN,NaN,DB/216 CMAN/46002,42.500000,-130.300000,3.0
31920,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99217,-,NaN,NaN,99217.0,NaN,NaN,DB/217 CMAN/46003,51.900000,-155.900000,3.0
31921,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99219,-,NaN,NaN,99219.0,NaN,NaN,DB/219 CMAN/46004,50.900000,-135.900000,3.0
31922,USA,US,United States,-,-,Buoy,Buoy,NaN,USs99220,-,NaN,NaN,99220.0,NaN,NaN,DB/220 CMAN/46006,40.800000,-137.600000,3.0


In [23]:
usa_stations = usa_stations[['country3', 'region', 'subregion', 'place_name', 'station_name', 'icao', 'lat', 'lon', 'elev']]

In [24]:
usa_stations.head()

,country3,region,subregion,place_name,station_name,icao,lat,lon,elev
31913,USA,NaN,NaN,"Albany, Ny.","Albany, Ny.",NaN,42.691944,-73.832222,95.0
31914,USA,-,-,Buoy,Buoy,NaN,34.900000,-72.900000,NaN
31915,USA,-,-,Buoy,Buoy,NaN,56.300000,-148.300000,3.0
31916,USA,-,-,Buoy,Buoy,NaN,25.900000,-89.700000,NaN
31917,USA,-,-,Buoy,Hotel,KC7H,38.500000,-70.700000,NaN


In [25]:
subset = usa_stations.loc[usa_stations['region'].isin(['AR','IL', 'IN', 'KY', 'ME', 'MO', 'NH', 'NY', 'OH', 'OK', 'PA', 'TX', 'VT'])].reset_index()

In [28]:
subset[['lat', 'lon']].replace('', np.nan, inplace=True)

/Users/noether/miniforge3/envs/eclipse/lib/python3.10/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [29]:
subset.dropna()
subset

,index,country3,region,subregion,place_name,station_name,icao,lat,lon,elev
0,32865,USA,AR,Arkansas County,Almyra|Holdridge,Almyra Muni,KM73,34.412328,-91.466347,64.1
1,32866,USA,AR,Clark County,Arkadelphia,Arkadelphia,KADF,34.099806,-93.066083,55.3
2,32867,USA,AR,Clark County,Arkadelphia,Arkadelphia,KM89,34.099806,-93.066083,55.5
3,32868,USA,AR,Sharp County,Ash Flat|Highland,Sharp County Regional AP,KCVK,36.264889,-91.562667,218.6
4,32869,USA,AR,Woodruff County,Augusta|Cavell,Woodruff County,KM60,35.271897,-91.269656,61.0
...,...,...,...,...,...,...,...,...,...,...
2227,39477,USA,VT,Caledonia County,St Johnsbury,St Johnsbury,K1V4,44.416667,-72.016667,203.0
2228,39478,USA,VT,Addison County,Vergennes|Rock Landing,Basin Harbor,KB06,44.195886,-73.349570,37.8
2229,39479,USA,VT,Washington County,Warren|East Warren,Warren-sugarbush,K0B7,44.116727,-72.827058,448.1
2230,39480,USA,VT,Windham County,West Dover,West Dover,NaN,42.933333,-72.866667,591.0


In [30]:
weather_stations = [Point(xy) for xy in zip(subset['lon'], subset['lat'])]
weather_stations[:3]

In [31]:
station_loc = gpd.GeoDataFrame(subset, crs = CRS("WGS84"), geometry = weather_stations)
station_loc.head()

,index,country3,region,subregion,place_name,station_name,icao,lat,lon,elev,geometry
0,32865,USA,AR,Arkansas County,Almyra|Holdridge,Almyra Muni,KM73,34.412328,-91.466347,64.1,POINT (-91.46635 34.41233)
1,32866,USA,AR,Clark County,Arkadelphia,Arkadelphia,KADF,34.099806,-93.066083,55.3,POINT (-93.06608 34.09981)
2,32867,USA,AR,Clark County,Arkadelphia,Arkadelphia,KM89,34.099806,-93.066083,55.5,POINT (-93.06608 34.09981)
3,32868,USA,AR,Sharp County,Ash Flat|Highland,Sharp County Regional AP,KCVK,36.264889,-91.562667,218.6,POINT (-91.56267 36.26489)
4,32869,USA,AR,Woodruff County,Augusta|Cavell,Woodruff County,KM60,35.271897,-91.269656,61.0,POINT (-91.26966 35.27190)


In [32]:
station_loc_json = station_loc.to_json()

In [33]:
test_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')
ids = folium.features.GeoJson(station_loc_json)

folium.GeoJson(station_loc_json, tooltip = folium.GeoJsonTooltip(fields=('icao', 'region', 'lat', 'lon')), show = True).add_to(test_map)
# test_map.add_child(ids)


folium.GeoJson(path).add_to(test_map)
folium.LatLngPopup().add_to(test_map)
test_map



TypeError: 'NoneType' object is not subscriptable

In [34]:
station_loc.geometry.is_empty.any()

True

In [41]:
empty = station_loc.geometry.is_empty
station_loc_non_empty = station_loc[~empty]
station_loc_non_empty.geometry

0       POINT (-91.46635 34.41233)
1       POINT (-93.06608 34.09981)
2       POINT (-93.06608 34.09981)
3       POINT (-91.56267 36.26489)
4       POINT (-91.26966 35.27190)
                   ...            
2227    POINT (-72.01667 44.41667)
2228    POINT (-73.34957 44.19589)
2229    POINT (-72.82706 44.11673)
2230    POINT (-72.86667 42.93333)
2231    POINT (-72.86565 42.92714)
Name: geometry, Length: 2227, dtype: geometry

In [42]:
station_loc_list = [[point.xy[1][0], point.xy[0][0]] for point in station_loc_nonempty.geometry]

NameError: name 'station_loc_nonempty' is not defined

In [43]:
test_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')

# Create a geometry list from the GeoDataFrame
station_loc_list = [[point.xy[1][0], point.xy[0][0]] for point in station_loc.geometry ]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in station_loc_list:
    #assign a color marker for the type of volcano, Strato being the most common
    if station_loc.region[i] == "AR":
        type_color = "green"
    elif station_loc.region[i] == "NY":
        type_color = "blue"
    elif station_loc.region[i] == "KY":
        type_color = "orange"
    elif station_loc.region[i] == "ME":
        type_color = "pink"
    else:
        type_color = "purple"


    # Place the markers with the popup labels and data
    test_map.add_child(folium.Marker(location = coordinates,
                            popup =
                            "Station ID: " + str(station_loc.icao[i]) + '<br>' +
                            "Station: " + str(station_loc.station_name[i]) + '<br>' +
                            "State: " + str(station_loc.region[i]) + '<br>'
                            "Elevation: " + str(station_loc.elev[i]) + '<br>'
                            "Coordinates: " + str(station_loc_list[i]),
                            icon = folium.Icon(color = "%s" % type_color)))
    i = i + 1

IndexError: array index out of range

In [38]:
locations = station_loc[['lon', 'lat']]
locationlist = locations.values.tolist()
locationlist

[[-91.46634722, 34.41232833],
 [-93.066083, 34.099806],
 [-93.06608333, 34.09980556],
 [-91.562667, 36.264889],
 [-91.26965556, 35.27189694],
 [-91.55769444, 35.29947222],
 [-91.6311, 35.7525],
 [-91.647444, 35.726222],
 [-91.7811, 35.82],
 [-94.219336, 36.34525],
 [-94.2, 36.3667],
 [-92.479444, 34.590389],
 [-93.62194, 36.32778],
 [-93.62455556, 36.38133333],
 [-89.830806, 35.940417],
 [-89.943944, 35.964333],
 [-93.86214167, 35.14951111],
 [-91.17644, 34.88027],
 [-92.14447222, 36.16452778],
 [-92.763389, 33.622806],
 [-92.300153, 34.850089],
 [-91.71211111, 34.80822222],
 [-91.39443806, 34.64797361],
 [-93.427155, 35.47069417],
 [-92.451583, 35.59775],
 [-92.40381294, 35.65071903],
 [-90.8265, 35.120056],
 [-92.42496167, 35.08080778],
 [-92.555111, 35.019889],
 [-92.33506697, 35.17752961],
 [-90.64792639, 36.40423139],
 [-91.880194, 33.178333],
 [-93.42751778, 35.08695667],
 [-94.399361, 34.047],
 [-94.28333333, 34.11666667],
 [-91.30748667, 34.26228306],
 [-94.44480833, 36.3435555

In [39]:
test_map = folium.Map([39.0119, -98.4842], zoom_start=4, tiles='cartodbpositron')

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=station_loc['icao'][point]).add_to(test_map)

ValueError: Location values cannot contain NaNs.